In [1]:
import os
import time
import sys
import cv2
import numpy as np
import torch
from huggingface_hub import login
from tqdm import tqdm

from transformers import (
    AutoTokenizer,
    BitsAndBytesConfig,
    PaliGemmaForConditionalGeneration,
    PaliGemmaProcessor,
)

hf_token = "<HF token here>"
login(token=hf_token)


In [2]:
device = torch.device("cuda")

# load the model
model_id = "google/paligemma-3b-mix-448"
model = PaliGemmaForConditionalGeneration.from_pretrained(
    model_id, torch_dtype=torch.float16, token=hf_token
)
processor = PaliGemmaProcessor.from_pretrained(model_id)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [3]:
def get_result(img, prompt):
    
    # Send text prompt and image as input.
    inputs = processor(
        text=prompt,
        images=img,
        padding="longest",
        do_convert_rgb=True,
        return_tensors="pt",
    ).to("cuda")

    model.to(device)
    inputs = inputs.to(dtype=model.dtype)

    # Get output
    with torch.no_grad():
        output = model.generate(**inputs, max_length=2048)

    paligemma_response = processor.decode(output[0], skip_special_tokens=True)[
        len(prompt) :
    ].lstrip("\n")

    return paligemma_response

In [4]:
prompt = "ocr"

opencv_img = cv2.cvtColor(
    cv2.imread("password.jpg"), cv2.COLOR_BGR2RGB
)

time_taken = [] 
for i in range(10):
    start_time = time.time()
    response = get_result(opencv_img, prompt)
    end_time = time.time()
    inference_time = end_time - start_time
    time_taken.append(inference_time)

print(response)
print(f"\n⏱️ Inference time: {np.average(time_taken):.3f} seconds")

You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.
You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.
You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.
Y

Ashley Hotel West Coast
WiFi Internet Service
Username: fqpp
Password: aaeu
Traffic: 1 GB
Price: 0.00
Validity: 1w
Starts: Now
Shared Users: 4
27/02/2019 11:03:15
To logout or check status please type in http://hello.wifi Thank you !

⏱️ Inference time: 3.108 seconds
